First, Python packages need to be imported for this project.

In [28]:
import numpy as np
import pandas as pd

Next, within the dataset chronicling all medals from all Olympic Games, those pertaining to 1960 through 2022 need to be selected.



In [88]:
# Initial data dump
col_list = ['discipline_title', 'slug_game', 'event_title', 'medal_type', 'country_name']
medal_table = pd.read_csv('archive/olympic_medals.csv', usecols = col_list)

# Extract columns for hosting city and year from "slug_game" column
medal_table['City'] = medal_table['slug_game'].str[:-4].str.replace('-', ' ').str.strip().str.title()
medal_table['Year'] = medal_table['slug_game'].str[-4:]

# Format event titles and medal types
medal_table['event_title'] = medal_table['event_title'].str.title()
medal_table['medal_type'] = medal_table['medal_type'].str.lower().str.title()

# Clean medal table with organized data
new_col_list = ['Year', 'City', 'discipline_title', 'event_title', 'medal_type', 'country_name']
new_medal_table = medal_table[new_col_list]

# Rename cleaned medal columns
new_medal_table = new_medal_table.rename(columns = {
    'discipline_title': 'Sport',
    'event_title': 'Event',
    'medal_type': 'Medal',
    'country_name': 'Country'
})

In [122]:
new_medal_table

,Year,City,Sport,Event,Medal,Country
0,2022,Beijing,Curling,Mixed Doubles,Gold,Italy
1,2022,Beijing,Curling,Mixed Doubles,Gold,Italy
2,2022,Beijing,Curling,Mixed Doubles,Silver,Norway
3,2022,Beijing,Curling,Mixed Doubles,Silver,Norway
4,2022,Beijing,Curling,Mixed Doubles,Bronze,Sweden
...,...,...,...,...,...,...
21692,1896,Athens,Weightlifting,Heavyweight - One Hand Lift Men,Silver,Denmark
21693,1896,Athens,Weightlifting,Heavyweight - One Hand Lift Men,Bronze,Greece
21694,1896,Athens,Weightlifting,Heavyweight - Two Hand Lift Men,Gold,Denmark
21695,1896,Athens,Weightlifting,Heavyweight - Two Hand Lift Men,Silver,Great Britain


The table is now compact and readable, but now needs only the rows that pertain to the Winter Olympics games from 1960 to 2022.

1960 &emsp; Squaw Valley, California, USA

1964 &emsp; Innsbruck, Austria

1968 &emsp; Grenoble, France

1972 &emsp; Sapporo, Japan

1976 &emsp; Innsbruck, Austria

1980 &emsp; Lake Placid, New York, USA

1984 &emsp; Sarajevo, Bosnia and Herzegovina (Yugoslavia)

1988 &emsp; Calgary, Alberta, Canada

1992 &emsp; Albertville, France

1994 &emsp; Lillehamer, Norway

1998 &emsp; Nagano, Japan

2002 &emsp;Salt Lake City, Utah, USA

2006 &emsp; Turin, Italy

2010 &emsp; Vancouver, British Columbia, Canada

2014 &emsp; Sochi, Russia

2018 &emsp; Pyeongchang, South Korea

2022 &emsp; Beijing, China

In [126]:
# List the years and cities for the Winter Olympic games of interest
winter_years = ['1960', '1964', '1968', '1972', '1976','1980', '1984', '1988', '1992', 
                '1994', '1998', '2002', '2006', '2010', '2014', '2018', '2022']
                
winter_cities = ['Squaw Valley', 'Innsbruck', 'Grenoble', 'Sapporo', 'Innsbruck', 
                 'Lake Placid', 'Sarajevo', 'Calgary', 'Albertille', 'Lillehamer',
                 'Nagano', 'Salt Lake City', 'Turin', 'Vancouver', 'Sochi', 'Pyeongchang',
                 'Beijing']

# Select the rows based on the above set of games
winter_medal_table = new_medal_table.loc[new_medal_table['Year'].isin(winter_years)]
winter_medal_table = winter_medal_table.loc[winter_medal_table['City'].isin(winter_cities)]

In [127]:
winter_medal_table

,Year,City,Sport,Event,Medal,Country
0,2022,Beijing,Curling,Mixed Doubles,Gold,Italy
1,2022,Beijing,Curling,Mixed Doubles,Gold,Italy
2,2022,Beijing,Curling,Mixed Doubles,Silver,Norway
3,2022,Beijing,Curling,Mixed Doubles,Silver,Norway
4,2022,Beijing,Curling,Mixed Doubles,Bronze,Sweden
...,...,...,...,...,...,...
16580,1960,Squaw Valley,Biathlon,20Km Men,Silver,Finland
16581,1960,Squaw Valley,Biathlon,20Km Men,Bronze,Soviet Union
16582,1960,Squaw Valley,Nordic Combined,Individual Men,Gold,Germany
16583,1960,Squaw Valley,Nordic Combined,Individual Men,Silver,Norway


In [130]:
winter_medal_table['Country'].value_counts()

Norway                                   280
United States of America                 277
Germany                                  251
Canada                                   205
Soviet Union                             201
Austria                                  190
Switzerland                              137
Sweden                                   136
Netherlands                              136
German Democratic Republic (Germany)     129
Finland                                  120
France                                   117
Russian Federation                       114
Italy                                    113
People's Republic of China                78
Republic of Korea                         69
Japan                                     63
Federal Republic of Germany               48
ROC                                       38
Czech Republic                            33
Great Britain                             23
Poland                                    22
Slovenia  

In [113]:
new_medal_table.loc[
        (new_medal_table['Year'] == [winter_games[x][0] for x in range(len(winter_games))])]

ValueError: ('Lengths must match to compare', (21697,), (17,))